This is the second fase.
Which is making, training and testing the model.
the data was preprocessed and are saved as .npy files in "processed_data" directory.
To check the preprocessing check the "PreProcessing.ipynb" notebook file.

# imports

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout